In [1]:
require './freegroup.rb'

Del = Word.new('abAB', 'cdCD', 's').freeze
'Del = ' + Del.show

DelPerms = []
perm = Del.dup
(Del.size).times do |n|
  perm = perm.cyclic_permutation
  DelPerms << perm << perm.inverse
end
DelPerms.freeze
DelPerms.map(&:show)

["bABcdCDsa", "ASdcDCbaB", "ABcdCDsab", "BASdcDCba", "BcdCDsabA", "aBASdcDCb", "cdCDsabAB", "baBASdcDC", "dCDsabABc", "CbaBASdcD", "CDsabABcd", "DCbaBASdc", "DsabABcdC", "cDCbaBASd", "sabABcdCD", "dcDCbaBAS", "abABcdCDs", "SdcDCbaBA"]

In [2]:
w = Word.new('abDdcdbABcd'); nil

### (i) Contracting the word

In [3]:
# (i) contract
reduced = w.contract.dup
reduced.show

"abcdbABcd"

### (ii) removing subwords

In [7]:
def reduce_once(myw)
  facs = myw.flatten.factors
  (myw.size+1).times do |i|
    w1 = Word.new(facs[0,i])
    w2 = Word.new(facs.drop(i))
    original_size = myw.size
    for perm in DelPerms
      wc = (w1*perm*w2).deep_copy.contract
      if (wc.size < original_size) then
        myw = wc 
        break
      end
    end
    if (myw.size < original_size) then
      return myw
      break
    end
  end
  return nil    
end
    
def reduce(myword)
  while not myword.nil?
    prev = myword
    myword = reduce_once(myword)
  end
  return prev.contract
end
nil

In [8]:
p 'w = ' + w.show + ' and w.size = ' + w.size.to_s
p reduce(w).show

"w = abcdbABcd and w.size = 9"
"abcdASdc"


"abcdASdc"

### (iii) removing subwords part2